**Module Paths**

In [1]:
import os

os.chdir("../../")

The app will show the top 5 classes that the model think are most relevant for the picture you have uploaded

In [2]:
from ipywidgets import VBox, Button, FileUpload, Output, Label
from PIL import Image
from IPython.display import display
import io
import numpy as np
import torchvision.transforms as T
import torch

# Load the model
learn_inf = torch.jit.load("checkpoints/transfer_exported.pt")


def on_click_classify(change):
    try:
        # Ensure there is a file uploaded
        if not btn_upload.value:
            print("No file uploaded.")
            return

        # Get the uploaded file content
        uploaded_file = list(btn_upload.value)[0]
        file_content = uploaded_file[
            "content"
        ]  # Get the content of the first file in the dictionary

        # Load the image from the file content
        fn = io.BytesIO(file_content)
        img = Image.open(fn)
        img.load()

        # Clear previous output
        out_pl.clear_output()

        # Display the image in the output
        with out_pl:
            ratio = img.size[0] / img.size[1]
            c = img.copy()
            c.thumbnail([ratio * 200, 200])
            display(c)

        # Transform image to tensor
        timg = T.ToTensor()(img).unsqueeze_(0)

        # Make a prediction
        softmax = learn_inf(timg).data.cpu().numpy().squeeze()

        # Get top 5 predictions
        idxs = np.argsort(softmax)[::-1][:5]

        for i, idx in enumerate(idxs):
            prob = softmax[idx]
            label_name = learn_inf.class_names[
                idx
            ]  # Accessing class names from the model
            labels[i].value = f"{label_name} (prob: {prob:.2f})"

    except Exception as e:
        out_pl.append_stdout(f"Error: {e}")


# File upload widget
btn_upload = FileUpload(accept="image/*", multiple=False)

# Button to run classification
btn_run = Button(description="Classify")
btn_run.on_click(on_click_classify)

# Labels to display top predictions
labels = [Label() for _ in range(5)]

# Output widget for image display
out_pl = Output()
out_pl.clear_output()

# VBox layout
wgs = [Label("Please upload a picture of a landmark"), btn_upload, btn_run, out_pl]
wgs.extend(labels)
display(VBox(wgs))